# Packages 

In [2]:
import Pkg;
#Pkg.add("DifferentialEquations")
#Pkg.add("Plots")
#Pkg.add("NLsolve")
#Pkg.add("ForwardDiff")
#Pkg.add("LinearAlgebra")
#Pkg.add("Interact")
#Pkg.add("JuMP")
#Pkg.add("Ipopt")
#Pkg.add("OptimalControl")
#Pkg.add("Roots")
# Packages
using DifferentialEquations
using Plots
using NLsolve
using ForwardDiff
using LinearAlgebra
using Roots
include("utils.jl"); # plot_traj!, plot_flow!, Flow
include("Fonctions.jl");



# Données 

In [3]:
b = 1
e = 1
a = 1


1

# Choix de variables :

In [4]:
t0 = 0
tf = 40
m0 = 0.0;


# Trouver u_bar, m_bar et p_bar : 

In [6]:
# Cas cholesky : 
Ψ(m) = ((a*m -b)^2 - 2*a*b*e - 2*b^2)/(4*((e+m)^4))
m_bar = find_zeros(Ψ, 0, 10)
m_bar = m_bar[1]
u_bar = (a*m_bar - b)/(a*m_bar + b);
p_bar = -2/ (b+ a*m_bar)
println("u_bar :", u_bar)
println("m_bar :",m_bar)
println("p_bar: ",p_bar )



u_bar :0.5
m_bar :3.0
p_bar: -0.5


# Tir simple: 

In [ ]:

# fonction de tir
function shoot(p0, t1, t2)
    # integration
    m1, p1 = fmin(t0, m0, p0, t1) # m1, p1 sont des scalaires
    m2, p2 = fsin(t1, m1, p1, t2)
    m3, p3 = fmax(t2, m2, p2, tf)
    
    # conditions
    s = zeros(eltype(p0), 3)
    s[1] = p3
    s[2] = 0.0
    s[3] = 0.0
    
    return s

end;
# itéré initiale pour la méthode indirecte de tir
p0 = -0.8## REMPLACER NaN par une bonne valeur
t1 = 7.5 ## REMPLACER NaN par une bonne valeur
t2 = 16.0 ## REMPLACER NaN par une bonne valeur

#
y = [ p0 ; t1 ; t2]

println("Itéré initial:\n", y)

# fonction de tir et sa jacobienne
foo(y) = shoot(y...)
jfoo(y) = ForwardDiff.jacobian(foo, y)

# Résolution de shoot(p0, t1, t2) = 0.
nl_sol1 = nlsolve(foo, jfoo, y; xtol=1e-6, method=:trust_region, show_trace=true);

# Retrieves solution
if converged(nl_sol1)
    p0 = nl_sol1.zero[1]
    t1 = nl_sol1.zero[2]
    t2 = nl_sol1.zero[3];
    println("\nFinal solution:\n", nl_sol1.zero)
else
    error("Not converged")
end



In [ ]:
x_plot = plot()
u_plot = plot()
px_plot = plot()
plt_traj_tir(x_plot, u_plot, px_plot,t0,t1,t2,m0,p0)
display(plot(x_plot, px_plot, layout = (1,2), size=(800, 300)))
display(u_plot)

# Tir multiple Sans la condition p_bar : 

In [ ]:


function shoot(p0, t1, t2)
    # integration
    m1, p1 = fmax(t0, m0, p0, t1) # m1, p1 sont des scalaires
    m2, p2 = fsin(t1, m1, p1, t2)
    m3, p3 = fmax(t2, m2, p2, tf)
    
    # conditions
    s = zeros(eltype(p0), 3)
    s[1] = (m1 - m_bar)^2 
    s[2] = (m2 - m_bar)^2 
    s[3] = p3
    
    return s

end;
p0 = -0.8## REMPLACER NaN par une bonne valeur
t1 = 8.0 ## REMPLACER NaN par une bonne valeur
t2 = 17.0## REMPLACER NaN par une bonne valeur

y = [ p0 ; t1 ; t2]

println("Itéré initial:\n", y)

# fonction de tir et sa jacobienne
foo(y) = shoot(y...)
jfoo(y) = ForwardDiff.jacobian(foo, y)

# Résolution de shoot(p0, t1, t2) = 0.
nl_sol3 = nlsolve(foo, jfoo, y; xtol=1e-6, method=:trust_region, show_trace=true);

# Retrieves solution
if converged(nl_sol3)
    p0 = nl_sol3.zero[1]
    t1 = nl_sol3.zero[2]
    t2 = nl_sol3.zero[3];
    println("\nFinal solution:\n", nl_sol3.zero)
else
    error("Not converged")
end



## Tir multiple avec condition p_bar :

In [ ]:

# fonction de tir
function shoot(p0, t1, t2)
    # integration
    m1, p1 = fmax(t0, m0, p0, t1) # m1, p1 sont des scalaires
    m2, p2 = fsin(t1, m1, p1, t2)
    m3, p3 = fmax(t2, m2, p2, tf)
    
    # conditions
    s = zeros(eltype(p0), 3)
    s[1] = (m1 - m_bar)^2 + (p1 - p_bar)^2
    s[2] = (m2 - m_bar)^2 + (p2 - p_bar)^2
    s[3] = p3
    
    return s

end;
# itéré initiale pour la méthode indirecte de tir
p0 = -0.8## REMPLACER NaN par une bonne valeur
t1 = 15.0 ## REMPLACER NaN par une bonne valeur
t2 = 17.0 ## REMPLACER NaN par une bonne valeur

#
y = [ p0 ; t1 ; t2]

println("Itéré initial:\n", y)

# fonction de tir et sa jacobienne
foo(y) = shoot(y...)
jfoo(y) = ForwardDiff.jacobian(foo, y)

# Résolution de shoot(p0, t1, t2) = 0.
nl_sol2 = nlsolve(foo, jfoo, y; xtol=1e-6, method=:trust_region, show_trace=true);

# Retrieves solution
if converged(nl_sol2)
    p0 = nl_sol2.zero[1]
    t1 = nl_sol2.zero[2]
    t2 = nl_sol2.zero[3];
    println("\nFinal solution:\n", nl_sol2.zero)
else
    error("Not converged")
end

println("Avec tir simple : ", nl_sol1.zero, "\navec Tir multiple avec condition p_bar: ", nl_sol2.zero,"\n avec Tir multiple sans condition p_bar", nl_sol3.zero)

In [ ]:
#Plots 
# affichage de la solution

ode_sol = fmax((t0, t1), m0, p0)

tt0 = ode_sol.t
xx0 = [ ode_sol[1, j] for j in 1:size(tt0, 1) ]
pp0 = [ ode_sol[2, j] for j in 1:size(tt0, 1) ]
uu0 = ones(size(tt0, 1))


ode_sol = fsin((t1, t2), xx0[end], pp0[end])
tt1 = ode_sol.t
xx1 = [ ode_sol[1, j] for j in 1:size(tt1, 1) ]
pp1 = [ ode_sol[2, j] for j in 1:size(tt1, 1) ]
uu1 = zeros(size(tt1, 1)) .+ u_bar

ode_sol = fmax((t2, tf), xx1[end], pp1[end])
tt2 = ode_sol.t
xx2 = [ ode_sol[1, j] for j in 1:size(tt2, 1) ]
pp2 = [ ode_sol[2, j] for j in 1:size(tt2, 1) ]
uu2 = ones(size(tt2, 1))

t_shoot = [ tt0 ; tt1 ; tt2 ]
x_shoot = [ xx0 ; xx1 ; xx2 ]
p_shoot = [ pp0 ; pp1 ; pp2 ]
u_shoot = [ uu0 ; uu1 ; uu2 ]    

x_plot = plot(t_shoot, x_shoot, xlabel = "t", ylabel = "m", legend = false)
u_plot = plot(t_shoot, u_shoot, xlabel = "t", ylabel = "u", legend = false, size=(800,300), linetype=:steppre)
px_plot = plot(t_shoot, p_shoot, xlabel = "t", ylabel = "px", legend = false)
display(plot(x_plot, px_plot, layout = (1,2), size=(800, 300)))
display(u_plot)


# Tir multiple pour chercher U_bar :

In [ ]:

# fonction de tir
function shoot(p0, t1, t2, u_bar2)
    hv_sin(m, p) = hv(m, p , u_bar2) ## Valeur u = u_bar 
    fsin = Flow(hv_sin)
    println("\nubar2 :",u_bar2)
    # integration
    m1, p1 = fmax(t0, m0, p0, t1) # m1, p1 sont des scalaires
    m2, p2 = fsin(t1, m1, p1, t2)
    m3, p3 = fmax(t2, m2, p2, tf)
    
    # conditions
    s = zeros(eltype(p0), 4)
    s[1] = m1 - m2 
    s[2] = commut(m1)
    s[3] = commut(m2)
    s[4] = p3
    
    return s

end;
# itéré initiale pour la méthode indirecte de tir
p0 = -0.8## REMPLACER NaN par une bonne valeur
t1 = 8.0 ## REMPLACER NaN par une bonne valeur
t2 = 16.0 ## REMPLACER NaN par une bonne valeur
u_bar2 = 0.5
#
y = [ p0 ; t1 ; t2 ; u_bar2]

println("Itéré initial:\n", y)

# fonction de tir et sa jacobienne
foo(y) = shoot(y...)
jfoo(y) = ForwardDiff.jacobian(foo, y)

# Résolution de shoot(p0, t1, t2) = 0.
nl_sol4 = nlsolve(foo, jfoo, y; xtol=1e-6, method=:trust_region, show_trace=true);

# Retrieves solution
if converged(nl_sol4)
    p0 = nl_sol4.zero[1]
    t1 = nl_sol4.zero[2]
    t2 = nl_sol4.zero[3]
    u_bar2 = nl_sol4.zero[4]
    println("\nFinal solution:\n", nl_sol4.zero)
else
    error("Not converged")
end

println("Avec tir simple : ", nl_sol1.zero, "\navec Tir multiple avec condition p_bar: ", nl_sol2.zero,"\n avec Tir multiple sans condition p_bar", nl_sol3.zero,
        "\nAvec tir multiple pour connaitre u_bar", nl_sol4.zero)

# Tir Multiple 

In [7]:


function shoot(p0, t1, t2)
    # integration
    m1, p1 = fmax(t0, m0, p0, t1) # m1, p1 sont des scalaires
    m2, p2 = fsin(t1, m1, p1, t2)
    m3, p3 = fmax(t2, m2, p2, tf)
    
    # conditions
    s = zeros(eltype(p0), 3)
    s[1] = (m1 - m_bar)^2 
    s[2] = (p2 - p_bar)^2 
    s[3] = p3
    
    return s

end;
p0 = -0.8## REMPLACER NaN par une bonne valeur
t1 = 8.0 ## REMPLACER NaN par une bonne valeur
t2 = 17.0## REMPLACER NaN par une bonne valeur

y = [ p0 ; t1 ; t2]

println("Itéré initial:\n", y)

# fonction de tir et sa jacobienne
foo(y) = shoot(y...)
jfoo(y) = ForwardDiff.jacobian(foo, y)

# Résolution de shoot(p0, t1, t2) = 0.
nl_sol3 = nlsolve(foo, jfoo, y; xtol=1e-6, method=:trust_region, show_trace=true);

# Retrieves solution
if converged(nl_sol3)
    p0 = nl_sol3.zero[1]
    t1 = nl_sol3.zero[2]
    t2 = nl_sol3.zero[3];
    println("\nFinal solution:\n", nl_sol3.zero)
else
    error("Not converged")
end



Itéré initial:
[-0.8, 8.0, 17.0]
Iter     f(x) inf-norm    Step 2-norm 
------   --------------   --------------
     0     1.050450e+00              NaN
     1     1.457659e+00     3.552714e-15
     2     1.457659e+00     0.000000e+00
     3     1.457659e+00     0.000000e+00
     4     5.459222e-01     9.566315e+00
     5     1.091448e-01     3.511340e+01
     6     7.320491e-01     0.000000e+00
     7     7.320491e-01     0.000000e+00
     8     1.158539e-01     0.000000e+00
     9     2.245046e-02     8.107990e+00
    10     1.377045e-01     0.000000e+00
    11     2.752054e-02     0.000000e+00
    12     8.048958e-03     3.913337e+00
    13     3.061799e-02     4.440892e-16
    14     3.924609e-03     3.958746e+00
    15     1.807683e-02     8.881784e-16
    16     8.626083e-03     0.000000e+00
    17     1.617763e-03     2.122591e+00
    18     9.199656e-03     0.000000e+00
    19     2.457829e-03     0.000000e+00
    20     1.207237e-03     1.084754e+00
    21     2.869817e-03   